In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
%matplotlib inline

In [92]:
# load dataset
data = pd.read_csv('./data/cardekho_imputated.csv')
data.head()
data.isnull().sum()
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [93]:
data.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [94]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [95]:
data.head()
data['model'].unique()
data['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [96]:
# get all the different type of columns
num_features = [feature for feature in data.columns if data[feature].dtype != 'object']
print(num_features)
cat_features = [feature for feature in data.columns if data[feature].dtype == 'object']
print(cat_features)

['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
['car_name', 'brand', 'model', 'seller_type', 'fuel_type', 'transmission_type']


In [97]:
# Divide dataset into dep and indep features
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Feature Encoding here

In [98]:
from sklearn.preprocessing import LabelEncoder

# we don't user one hot encode cause it make extra 120 columns for 120 unique values
encoder = LabelEncoder()
data['model'] = encoder.fit_transform(data['model'])

In [99]:
cat_features
data['seller_type'].unique(), data['fuel_type'].unique(), data['transmission_type'].unique()

(array(['Individual', 'Dealer', 'Trustmark Dealer'], dtype=object),
 array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object),
 array(['Manual', 'Automatic'], dtype=object))

In [100]:
num_features = X.select_dtypes(exclude='object').columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transforamtion = StandardScaler()
oh_tranformation = OneHotEncoder(drop='first')

transformer = ColumnTransformer(
    [
        ("OneHotEncoder", oh_tranformation, onehot_columns),
        ("StandardScaler", num_transforamtion, num_features),
    ], remainder='passthrough'  # remainder='passthrough' means other features not chnages or delete without we need
)

In [101]:
X = transformer.fit_transform(X)

In [106]:
X

array([[1.0, 0.0, 0.0, ..., 'Maruti Alto', 'Maruti', 'Alto'],
       [1.0, 0.0, 0.0, ..., 'Hyundai Grand', 'Hyundai', 'Grand'],
       [1.0, 0.0, 0.0, ..., 'Hyundai i20', 'Hyundai', 'i20'],
       ...,
       [0.0, 0.0, 1.0, ..., 'Skoda Rapid', 'Skoda', 'Rapid'],
       [0.0, 0.0, 1.0, ..., 'Mahindra XUV500', 'Mahindra', 'XUV500'],
       [0.0, 0.0, 0.0, ..., 'Honda City', 'Honda', 'City']],
      shape=(15411, 16), dtype=object)

# Train Test Split

In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size= 0.2,
    random_state=42
)

# Model selection and training the Model

In [103]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [104]:
def evalute(true, prediction):
    mae = mean_absolute_error(true, prediction)
    mse = mean_squared_error(true, prediction)
    rmse = np.sqrt(mean_absolute_error(true, prediction))
    r2_score = r2_score(true, prediction)
    
    return mae, mse, rmse, r2_score

In [105]:
# Beggining the model
models= {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree Regressor" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    # make predictions
    y_train_pred = model.fit(X_train)
    y_test_pred = model.fit(X_test)
    
    # evaluete the performance
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2_score = evalute(y_tran, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2_score = evalute(y_test, y_test_pred)
    
    print(f"-------------model name: {name}--------------")
    print(f"Train - MAE: {model_train_mae}, MSE: {model_train_mse}, RMSE: {model_train_rmse}, R2: {model_train_r2_score}")
    print(f"Test  - MAE: {model_test_mae}, MSE: {model_test_mse}, RMSE: {model_test_rmse}, R2: {model_test_r2_score}\n")
    
    print('------' * 20)
    

ValueError: could not convert string to float: 'Volvo XC60'

AttributeError: 'numpy.ndarray' object has no attribute 'head'